In [2]:
# fit_tripp.py
import numpy as np
from scipy import optimize, integrate

# -----------------------------
# User settings (edit as needed)
# -----------------------------
FILENAME = "/Users/pittsburghgraduatestudent/repos/myc21_ztf_mu/ZTF_snia_data.csv"   # your CSV path
H0 = 70.0                 # km/s/Mpc
OMEGA_M = 0.3
SIGMA_INT = 0.10          # mag intrinsic scatter (set 0.0 to ignore)

# -----------------------------
# Helpers
# -----------------------------
C_KMS = 299792.458
LN10 = np.log(10.0)

def read_ztf_csv(path):
    """
    Reads the ZTF-like CSV you pasted. It tolerates a leading index column.
    Returns arrays: z, x0, x1, c, sig_x0, sig_x1, sig_c, cov_x0_x1, cov_x0_c, cov_x1_c
    """
    # Load with names so we can refer to columns by header labels
    data = np.genfromtxt(path, delimiter=",", names=True, dtype=None, encoding="utf-8")

    # If there is an unlabeled first column, drop it by detecting known names
    names = data.dtype.names

    def get(name):
        if name in names:
            return np.array(data[name], dtype=float)
        raise KeyError(f"Column '{name}' not found in CSV. Present columns: {names}")

    z = get("redshift")
    x0 = get("x0")
    x1 = get("x1")
    c  = get("c")

    sig_x0 = get("x0_err")
    sig_x1 = get("x1_err")
    sig_c  = get("c_err")

    # Covariance columns can be small/zero; present in your sample
    cov_x0_x1 = get("cov_x0_x1")
    cov_x0_c  = get("cov_x0_c")
    cov_x1_c  = get("cov_x1_c")

    return z, x0, x1, c, sig_x0, sig_x1, sig_c, cov_x0_x1, cov_x0_c, cov_x1_c

def E_of_z(z, omega_m=OMEGA_M):
    return np.sqrt(omega_m*(1+z)**3 + (1.0-omega_m))

def lum_dist_Mpc(z, H0=H0, omega_m=OMEGA_M):
    """
    Flat ΛCDM luminosity distance in Mpc: D_L = (1+z) * (c/H0) * \int_0^z dz'/E(z')
    """
    if np.isscalar(z):
        integ, _ = integrate.quad(lambda zp: 1.0/E_of_z(zp, omega_m), 0.0, z, epsrel=1e-7)
        return (1.0 + z) * (C_KMS/H0) * integ
    # vectorized
    out = np.empty_like(z, dtype=float)
    for i, zi in enumerate(z):
        integ, _ = integrate.quad(lambda zp: 1.0/E_of_z(zp, omega_m), 0.0, zi, epsrel=1e-7)
        out[i] = (1.0 + zi) * (C_KMS/H0) * integ
    return out

def mu_LCDM(z):
    dL = lum_dist_Mpc(z)
    return 5.0*np.log10(dL) + 25.0

def mu_obs(alpha, beta, MB, x0, x1, c):
    return (-2.5*np.log10(x0) + alpha*x1 - beta*c + MB)

def sigma_mu_meas2(alpha, beta,
                   x0, x1, c,
                   sig_x0, sig_x1, sig_c,
                   cov_x0_x1, cov_x0_c, cov_x1_c):
    """
    Propagate SALT2 parameter uncertainties into Var(mu_obs).
    mu_obs = -2.5*log10(x0) + alpha*x1 - beta*c + M_B
    """
    dmu_dx0 = -2.5 / (x0 * LN10)
    dmu_dx1 =  alpha
    dmu_dc  = -beta

    var = (dmu_dx0**2) * (sig_x0**2) \
        + (dmu_dx1**2) * (sig_x1**2) \
        + (dmu_dc **2) * (sig_c **2) \
        + 2.0 * dmu_dx0 * dmu_dx1 * cov_x0_x1 \
        + 2.0 * dmu_dx0 * dmu_dc  * cov_x0_c  \
        + 2.0 * dmu_dx1 * dmu_dc  * cov_x1_c
    return var

def chisq_tripp(params, z, x0, x1, c,
                sig_x0, sig_x1, sig_c,
                cov_x0_x1, cov_x0_c, cov_x1_c,
                sigma_int=SIGMA_INT):
    alpha, beta, MB = params
    mu_model = mu_LCDM(z)
    mu_meas  = mu_obs(alpha, beta, MB, x0, x1, c)

    var_meas = sigma_mu_meas2(alpha, beta, x0, x1, c,
                              sig_x0, sig_x1, sig_c,
                              cov_x0_x1, cov_x0_c, cov_x1_c)
    var_tot = var_meas + sigma_int**2
    chi2 = np.sum((mu_meas - mu_model)**2 / var_tot)
    return chi2

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    # Load data
    (z, x0, x1, c,
     sig_x0, sig_x1, sig_c,
     cov_x0_x1, cov_x0_c, cov_x1_c) = read_ztf_csv(FILENAME)

    # Initial guess near typical literature values
    x0_init = np.array([0.14, 3.1, -19.3])  # [alpha, beta, M_B]

    # Minimize chi^2
    res = optimize.minimize(
        chisq_tripp, x0_init,
        args=(z, x0, x1, c,
              sig_x0, sig_x1, sig_c,
              cov_x0_x1, cov_x0_c, cov_x1_c,
              SIGMA_INT),
        method="Nelder-Mead"
    )

    # Optionally refine with a local Hessian-based method for errors
    res2 = optimize.minimize(
        chisq_tripp, res.x,
        args=(z, x0, x1, c,
              sig_x0, sig_x1, sig_c,
              cov_x0_x1, cov_x0_c, cov_x1_c,
              SIGMA_INT),
        method="BFGS"
    )

    best = res2.x
    try:
        cov = res2.hess_inv
        errs = np.sqrt(np.diag(cov))
    except Exception:
        cov = None
        errs = [np.nan, np.nan, np.nan]

    print("\n=== Tripp fit (flat ΛCDM H0=%.1f, Ωm=%.2f, σ_int=%.3f) ===" % (H0, OMEGA_M, SIGMA_INT))
    print("alpha = %.6f ± %.6f" % (best[0], errs[0]))
    print("beta  = %.6f ± %.6f" % (best[1], errs[1]))
    print("M_B   = %.6f ± %.6f" % (best[2], errs[2]))
    print("chi2/dof ~ %.2f" % (res2.fun / max(1, len(z)-len(best))))

ValueError: Some errors were detected !
    Line #2 (got 31 columns instead of 30)
    Line #3 (got 31 columns instead of 30)
    Line #4 (got 31 columns instead of 30)
    Line #5 (got 31 columns instead of 30)
    Line #6 (got 31 columns instead of 30)
    Line #7 (got 31 columns instead of 30)
    Line #8 (got 31 columns instead of 30)
    Line #9 (got 31 columns instead of 30)
    Line #10 (got 31 columns instead of 30)
    Line #11 (got 31 columns instead of 30)
    Line #12 (got 31 columns instead of 30)
    Line #13 (got 31 columns instead of 30)
    Line #14 (got 31 columns instead of 30)
    Line #15 (got 31 columns instead of 30)
    Line #16 (got 31 columns instead of 30)
    Line #17 (got 31 columns instead of 30)
    Line #18 (got 31 columns instead of 30)
    Line #19 (got 31 columns instead of 30)
    Line #20 (got 31 columns instead of 30)
    Line #21 (got 31 columns instead of 30)
    Line #22 (got 31 columns instead of 30)
    Line #23 (got 31 columns instead of 30)
